# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f7c58d962b0>
├── 'a' --> tensor([[-0.2160,  1.5278,  0.3153],
│                   [ 0.8149, -0.7797, -0.9736]])
└── 'x' --> <FastTreeValue 0x7f7cfc28bfd0>
    └── 'c' --> tensor([[-0.7709, -0.5655,  0.1681, -1.1895],
                        [-1.2836, -0.1488, -0.0166,  1.1525],
                        [-1.0072, -0.7077,  0.1132, -0.4709]])

In [4]:
t.a

tensor([[-0.2160,  1.5278,  0.3153],
        [ 0.8149, -0.7797, -0.9736]])

In [5]:
%timeit t.a

74.4 ns ± 0.63 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f7c58d962b0>
├── 'a' --> tensor([[ 0.1729,  0.9024,  0.5930],
│                   [ 0.7142,  0.2882, -0.9651]])
└── 'x' --> <FastTreeValue 0x7f7cfc28bfd0>
    └── 'c' --> tensor([[-0.7709, -0.5655,  0.1681, -1.1895],
                        [-1.2836, -0.1488, -0.0166,  1.1525],
                        [-1.0072, -0.7077,  0.1132, -0.4709]])

In [7]:
%timeit t.a = new_value

83.1 ns ± 1.73 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.2160,  1.5278,  0.3153],
               [ 0.8149, -0.7797, -0.9736]]),
    x: Batch(
           c: tensor([[-0.7709, -0.5655,  0.1681, -1.1895],
                      [-1.2836, -0.1488, -0.0166,  1.1525],
                      [-1.0072, -0.7077,  0.1132, -0.4709]]),
       ),
)

In [10]:
b.a

tensor([[-0.2160,  1.5278,  0.3153],
        [ 0.8149, -0.7797, -0.9736]])

In [11]:
%timeit b.a

66.9 ns ± 0.989 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.5718,  0.5578, -1.5466],
               [-0.1554, -0.2610, -0.2579]]),
    x: Batch(
           c: tensor([[-0.7709, -0.5655,  0.1681, -1.1895],
                      [-1.2836, -0.1488, -0.0166,  1.1525],
                      [-1.0072, -0.7077,  0.1132, -0.4709]]),
       ),
)

In [13]:
%timeit b.a = new_value

617 ns ± 8.44 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.01 µs ± 12.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.3 µs ± 203 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

293 µs ± 5.72 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

275 µs ± 9.81 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f7c58b2cc10>
├── 'a' --> tensor([[[-0.2160,  1.5278,  0.3153],
│                    [ 0.8149, -0.7797, -0.9736]],
│           
│                   [[-0.2160,  1.5278,  0.3153],
│                    [ 0.8149, -0.7797, -0.9736]],
│           
│                   [[-0.2160,  1.5278,  0.3153],
│                    [ 0.8149, -0.7797, -0.9736]],
│           
│                   [[-0.2160,  1.5278,  0.3153],
│                    [ 0.8149, -0.7797, -0.9736]],
│           
│                   [[-0.2160,  1.5278,  0.3153],
│                    [ 0.8149, -0.7797, -0.9736]],
│           
│                   [[-0.2160,  1.5278,  0.3153],
│                    [ 0.8149, -0.7797, -0.9736]],
│           
│                   [[-0.2160,  1.5278,  0.3153],
│                    [ 0.8149, -0.7797, -0.9736]],
│           
│                   [[-0.2160,  1.5278,  0.3153],
│                    [ 0.8149, -0.7797, -0.9736]]])
└── 'x' --> <FastTreeValue 0x7f7c58b2c9a0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

48.8 µs ± 2.59 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f7cfc2111f0>
├── 'a' --> tensor([[-0.2160,  1.5278,  0.3153],
│                   [ 0.8149, -0.7797, -0.9736],
│                   [-0.2160,  1.5278,  0.3153],
│                   [ 0.8149, -0.7797, -0.9736],
│                   [-0.2160,  1.5278,  0.3153],
│                   [ 0.8149, -0.7797, -0.9736],
│                   [-0.2160,  1.5278,  0.3153],
│                   [ 0.8149, -0.7797, -0.9736],
│                   [-0.2160,  1.5278,  0.3153],
│                   [ 0.8149, -0.7797, -0.9736],
│                   [-0.2160,  1.5278,  0.3153],
│                   [ 0.8149, -0.7797, -0.9736],
│                   [-0.2160,  1.5278,  0.3153],
│                   [ 0.8149, -0.7797, -0.9736],
│                   [-0.2160,  1.5278,  0.3153],
│                   [ 0.8149, -0.7797, -0.9736]])
└── 'x' --> <FastTreeValue 0x7f7c51f454f0>
    └── 'c' --> tensor([[-0.7709, -0.5655,  0.1681, -1.1895],
                        [-1.2836, -0.1488, -0.0166,  1.1525],
                 

In [23]:
%timeit t_cat(trees)

48.5 µs ± 3.21 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

81.3 µs ± 2.5 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.7709, -0.5655,  0.1681, -1.1895],
                       [-1.2836, -0.1488, -0.0166,  1.1525],
                       [-1.0072, -0.7077,  0.1132, -0.4709]],
              
                      [[-0.7709, -0.5655,  0.1681, -1.1895],
                       [-1.2836, -0.1488, -0.0166,  1.1525],
                       [-1.0072, -0.7077,  0.1132, -0.4709]],
              
                      [[-0.7709, -0.5655,  0.1681, -1.1895],
                       [-1.2836, -0.1488, -0.0166,  1.1525],
                       [-1.0072, -0.7077,  0.1132, -0.4709]],
              
                      [[-0.7709, -0.5655,  0.1681, -1.1895],
                       [-1.2836, -0.1488, -0.0166,  1.1525],
                       [-1.0072, -0.7077,  0.1132, -0.4709]],
              
                      [[-0.7709, -0.5655,  0.1681, -1.1895],
                       [-1.2836, -0.1488, -0.0166,  1.1525],
                       [-1.0072, -0.7077,  0.1132, -0.4709]],

In [26]:
%timeit Batch.stack(batches)

108 µs ± 3.58 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.7709, -0.5655,  0.1681, -1.1895],
                      [-1.2836, -0.1488, -0.0166,  1.1525],
                      [-1.0072, -0.7077,  0.1132, -0.4709],
                      [-0.7709, -0.5655,  0.1681, -1.1895],
                      [-1.2836, -0.1488, -0.0166,  1.1525],
                      [-1.0072, -0.7077,  0.1132, -0.4709],
                      [-0.7709, -0.5655,  0.1681, -1.1895],
                      [-1.2836, -0.1488, -0.0166,  1.1525],
                      [-1.0072, -0.7077,  0.1132, -0.4709],
                      [-0.7709, -0.5655,  0.1681, -1.1895],
                      [-1.2836, -0.1488, -0.0166,  1.1525],
                      [-1.0072, -0.7077,  0.1132, -0.4709],
                      [-0.7709, -0.5655,  0.1681, -1.1895],
                      [-1.2836, -0.1488, -0.0166,  1.1525],
                      [-1.0072, -0.7077,  0.1132, -0.4709],
                      [-0.7709, -0.5655,  0.1681, -1.1895],
                   

In [28]:
%timeit Batch.cat(batches)

189 µs ± 3.67 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

668 µs ± 74.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
